# Parens

In [1]:
parensGen :: Int -> Int -> String -> [String]
parensGen 0 0 s = [ s ]
parensGen (-1) _ _ = []
parensGen _ (-1) _ = []
parensGen openCount closedCount prefix =
    parensGen (openCount - 1) (closedCount + 1) (prefix ++ "(") ++
    parensGen openCount (closedCount - 1) (prefix ++ ")")

In [2]:
parens :: Int -> [String]
parens x = parensGen x 0 ""

In [3]:
parens 3

["((()))","(()())","(())()","()(())","()()()"]

Invariants:
* Combinations are valid
* Combinations contain requested number of parens
* There are no duplicate combinations
* The number of combinations matches 